## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model_Xception = keras.applications.xception.Xception(include_top = True, weights = 'imagenet', input_tensor = None, input_shape = None, pooling = None, classes=1000)
print(model_Xception.summary())

'''Resnet 50 架構'''
model_ResNet50 = keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_tensor = input_tensor, input_shape = (32, 32, 3), classes=10)
print(model_ResNet50.summary())


            
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 4, 4, 512)    0           add_19[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 2, 2, 256)    131328      activation_22[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 2, 2, 256)    1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 2, 2, 256)    0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 2, 2, 256)    590080      activation_23[0][0]            

## 添加層數

In [3]:
x = model_ResNet50.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim = 30, activation='relu')(x)
x = Dense(output_dim = 20, activation='relu')(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model_ResNet50 = Model(inputs = model_ResNet50.input, outputs=predictions)
print('Model深度：', len(model_ResNet50.layers))


Model深度： 179


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model_ResNet50.layers[:100]:
    layer.trainable = False
for layer in model_ResNet50.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model_ResNet50.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_ResNet50.fit(x_train,y_train, batch_size = 32, epochs = 10)

Epoch 1/10
50000/50000 [==============================] - 164s 3ms/step - loss: 1.2445 - accuracy: 0.5839
Epoch 2/10
50000/50000 [==============================] - 153s 3ms/step - loss: 0.9536 - accuracy: 0.6758
Epoch 3/10
50000/50000 [==============================] - 154s 3ms/step - loss: 0.7975 - accuracy: 0.7296
Epoch 4/10
50000/50000 [==============================] - 153s 3ms/step - loss: 0.7353 - accuracy: 0.7481
Epoch 5/10
50000/50000 [==============================] - 153s 3ms/step - loss: 0.7186 - accuracy: 0.7514
Epoch 6/10
50000/50000 [==============================] - 153s 3ms/step - loss: 0.6621 - accuracy: 0.7719
Epoch 7/10
50000/50000 [==============================] - 154s 3ms/step - loss: 0.6287 - accuracy: 0.7821
Epoch 8/10
50000/50000 [==============================] - 153s 3ms/step - loss: 0.5507 - accuracy: 0.8109
Epoch 9/10
50000/50000 [==============================] - 154s 3ms/step - loss: 0.4768 - accuracy: 0.8336
Epoch 10/10
50000/50000 [=====================